In [412]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = '/Users/bruno/Documents/Statistical_Analyst_Products/combined_ratings.csv'

# Read the CSV file
df = pd.read_csv(file_path)
df['Time'] = pd.to_datetime(df['Review - Time'], unit='s').dt.strftime('%w-%m-%Y')
df['Rating First Digit'] = df['Review - Overall'].astype(str).str[0].astype(int)
df = df[df['File Name'].str.contains("RateBeer", case=False)]

# Print the first few rows of the DataFrame
print(df.head())

/var/folders/88/g84kk_9n2498jwvfh8mnxj8w0000gn/T/ipykernel_60796/2282718759.py:7: DtypeWarning: Columns (2,4,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


         Unnamed: 0                                Name Beer ID  Brewer ID  \
1586614           0            John Harvards Simcoe IPA   63836       8481   
1586615           1            John Harvards Simcoe IPA   63836       8481   
1586616           2       John Harvards Cristal Pilsner   71716       8481   
1586617           3  John Harvards Fancy Lawnmower Beer   64125       8481   
1586618           4  John Harvards Fancy Lawnmower Beer   64125       8481   

         ABV                    Beer Style Review - Appearance Review - Aroma  \
1586614  5.4  India Pale Ale &#40;IPA&#41;                 4/5           6/10   
1586615  5.4  India Pale Ale &#40;IPA&#41;                 4/5           6/10   
1586616    5             Bohemian Pilsener                 4/5           5/10   
1586617  5.4                        Kölsch                 2/5           4/10   
1586618  5.4                        Kölsch                 2/5           4/10   

        Review - Palate Review - Taste Revie

In [413]:
df = df.head(100000)

In [414]:
#print(len(df)) #4.5M using both Filenames
print(len(df)) #2.9 using only RateBeer

100000


In [415]:
print(df.columns)

Index(['Unnamed: 0', 'Name', 'Beer ID', 'Brewer ID', 'ABV', 'Beer Style',
       'Review - Appearance', 'Review - Aroma', 'Review - Palate',
       'Review - Taste', 'Review - Overall', 'Review - Time',
       'Review - Profile Name', 'Review - Text', 'File Name', 'Time',
       'Rating First Digit'],
      dtype='object')


In [417]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

In [418]:
########## BrewerID & aux variables
df3 = pd.DataFrame(df['Brewer ID'])
df3['Unnamed: 0'] = df['Unnamed: 0']
df3['Time'] = df['Time']
df3['Beer Style'] = df['Beer Style']
df3['Rating First Digit'] = df['Rating First Digit']

print(df3.head())

         Brewer ID  Unnamed: 0       Time                    Beer Style  \
1586614       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1586615       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
1586616       8481           2  5-05-2000             Bohemian Pilsener   
1586617       8481           3  4-09-2006                        Kölsch   
1586618       8481           4  4-09-2006                        Kölsch   

         Rating First Digit  
1586614                   1  
1586615                   1  
1586616                   1  
1586617                   8  
1586618                   8  


In [419]:
########## Time
# The format '%w-%m-%Y' represents the day of the week, month, and year
df3['Time'] = pd.to_datetime(df['Review - Time'], unit='s').dt.strftime('%w-%m-%Y')

print("column Time was sucesfully created")
print(df3.head())

column Time was sucesfully created
         Brewer ID  Unnamed: 0       Time                    Beer Style  \
1586614       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1586615       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
1586616       8481           2  5-05-2000             Bohemian Pilsener   
1586617       8481           3  4-09-2006                        Kölsch   
1586618       8481           4  4-09-2006                        Kölsch   

         Rating First Digit  
1586614                   1  
1586615                   1  
1586616                   1  
1586617                   8  
1586618                   8  


In [420]:
########## NewProduct Calculation
# Find the first review date for each product ('Beer ID') in the original DataFrame
first_review = df.groupby('Beer ID')['Review - Time'].min().reset_index()
print(first_review.head())

first_review['FirstReviewTime'] = pd.to_datetime(first_review['Review - Time'], unit='s').dt.strftime('%w-%m-%Y')

print(first_review.head())

# Merge the first review dates with the 'Beer ID', 'Brewer ID', and 'Beer Style' from the original DataFrame
# This associates each first review with its corresponding product and brewer
# 'drop_duplicates' is used to ensure each 'Beer ID' is only counted once
first_review = first_review.merge(df[['Beer ID', 'Brewer ID', 'Beer Style']], on='Beer ID', how='left').drop_duplicates('Beer ID')

print(first_review.head())

  Beer ID  Review - Time
0      51      956707200
1      52      956707200
2      53      957052800
3      56      961027200
4      57     1042416000
  Beer ID  Review - Time FirstReviewTime
0      51      956707200       3-04-2000
1      52      956707200       3-04-2000
2      53      957052800       0-04-2000
3      56      961027200       4-06-2000
4      57     1042416000       1-01-2003
     Beer ID  Review - Time FirstReviewTime  Brewer ID          Beer Style
0         51      956707200       3-04-2000         12        Abbey Dubbel
2383      52      956707200       3-04-2000         12        Abbey Tripel
4658      53      957052800       0-04-2000         12  Belgian Strong Ale
7714      56      961027200       4-06-2000         14           Brown Ale
7907      57     1042416000       1-01-2003         14            Pilsener


In [421]:
# Count new products for each 'Brewer ID' in each time period
# This gives the number of new products introduced by each brewer in each time period
new_product_count = first_review.groupby(['Brewer ID', 'FirstReviewTime']).size().reset_index(name='NewProduct')
print("aca")
print(new_product_count.head())

aca
   Brewer ID FirstReviewTime  NewProduct
0          1       0-01-2010           2
1          1       0-02-2010           1
2          1       0-05-2000           1
3          1       0-06-2007           1
4          1       0-07-2006           1


In [422]:
# Merge the new product count with df3, matching on both 'BrewerID' and 'Time'
df3 = df3.merge(new_product_count, left_on=['Brewer ID', 'Time'], right_on=['Brewer ID', 'FirstReviewTime'], how='left')

# Drop unnecessary columns and handle missing data by filling NaNs with 0
df3['NewProduct'].fillna(0, inplace=True)
df3['NewProduct'] = df3['NewProduct'].apply(lambda x: 1 if x != 0 else 0)

print("column NewProduct was sucesfully created")
print(df3.head())

column NewProduct was sucesfully created
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit FirstReviewTime  NewProduct  
0                   1       4-09-2006           1  
1                   1       0-09-2006           1  
2                   1       5-05-2000           1  
3                   8       4-09-2006           1  
4                   8       4-09-2006           1  


In [423]:
print(df3.head())

   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit FirstReviewTime  NewProduct  
0                   1       4-09-2006           1  
1                   1       0-09-2006           1  
2                   1       5-05-2000           1  
3                   8       4-09-2006           1  
4                   8       4-09-2006           1  


In [424]:
########## NewMarket Calculation
# For each 'BeerID', determine if its first review is also the first entry of the brewer into that 'Beer Style'
# This step identifies first market entries into new beer styles for each brewer
first_style_entry = first_review.drop_duplicates(subset=['Brewer ID', 'Beer Style'])

# Count these first entries for each 'Brewer ID' and time period
# This gives the number of new market segments (beer styles) each brewer entered in each time period
new_market_count = first_style_entry.groupby(['Brewer ID', 'FirstReviewTime']).size().reset_index(name='NewMarket')

# Merge the new market count with df3, matching on both 'BrewerID' and 'Time'
df3 = df3.merge(new_market_count, left_on=['Brewer ID', 'Time'], right_on=['Brewer ID', 'FirstReviewTime'], how='left')

# Drop unnecessary columns and handle missing data by filling NaNs with 0
df3['NewMarket'].fillna(0, inplace=True)

df3['NewMarket'] = df3['NewMarket'].apply(lambda x: 1 if x != 0 else 0)
df3.drop(columns=['FirstReviewTime_x'], inplace=True)
df3 = df3.rename(columns={'FirstReviewTime_y': 'FirstReviewTime'})



In [425]:
print(df3.head())
print(len(df3))

   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  NewProduct FirstReviewTime  NewMarket  
0                   1           1       4-09-2006          1  
1                   1           1       0-09-2006          1  
2                   1           1       5-05-2000          1  
3                   8           1       4-09-2006          1  
4                   8           1       4-09-2006          1  
100000


In [426]:
########## NewCategory Calculation
# Identify the first occurrence of each BeerStyle
first_style_occurrence = df.groupby('Beer Style')['Review - Time'].min().reset_index()
first_style_occurrence['FirstStyleTime'] = pd.to_datetime(first_style_occurrence['Review - Time'], unit='s').dt.strftime('%w-%m-%Y')

# Count new BeerStyle categories for each time period
# This gives the number of new beer styles introduced in each time period
new_category_count = first_style_occurrence.groupby('FirstStyleTime').size().reset_index(name='NewCategory')

# Merge this count with df3
df3 = df3.merge(new_category_count, left_on='Time', right_on='FirstStyleTime', how='left')

# Drop the extra FirstStyleTime column
df3.drop('FirstStyleTime', axis=1, inplace=True)

# Handle missing data (if there are time periods with no new categories)
df3['NewCategory'].fillna(0, inplace=True)
df3['NewCategory'] = df3['NewCategory'].apply(lambda x: 1 if x != 0 else 0)

print("column NewCategory was sucesfully created")

column NewCategory was sucesfully created


In [431]:
########## Imitation Calculation

# Filter df3 for new market entries
new_market_entries = df3[df3['NewMarket'] > 0]

# Adjust the grouping in `product_launches`
product_launches = new_market_entries.groupby(['Beer Style', 'Time', 'Brewer ID']).size().reset_index(name='NewMarketEntryCount')

# Define the total number of rows in the DataFrame
total_rows = len(product_launches)

# Set an interval for printing progress updates
print_interval = 100  # Modify this number as needed

def calculate_imitation(brewer_id, beer_style, time, product_launches_df):
    time_window_start = datetime.strptime(time, '%w-%m-%Y') - relativedelta(months=3)
    time_window_start_str = time_window_start.strftime('%w-%m-%Y')

    competitor_launches = product_launches_df[
        (product_launches_df['Beer Style'] == beer_style) & 
        (product_launches_df['Time'] >= time_window_start_str) &
        (product_launches_df['Time'] < time) &
        (product_launches_df['Brewer ID'] != brewer_id)
    ]
    return competitor_launches['NewMarketEntryCount'].sum()

# Create a column for Imitation and print progress
product_launches['Imitation'] = 0
for index, row in product_launches.iterrows():
    imitation_count = calculate_imitation(row['Brewer ID'], row['Beer Style'], row['Time'], product_launches)
    product_launches.at[index, 'Imitation'] = 1 if imitation_count > 0 else 0
    if (index + 1) % print_interval == 0:
        print(f"Progress: {index + 1}/{total_rows}")

# Merge the Imitation results back into df3
df3 = df3.merge(product_launches[['Beer Style', 'Time', 'Brewer ID', 'Imitation']], 
                on=['Beer Style', 'Time', 'Brewer ID'], 
                how='left')

print("column Imitation was sucesfully created")
print(len(df3))
print(df3.head())

Progress: 100/5409
Progress: 200/5409
Progress: 300/5409
Progress: 400/5409
Progress: 500/5409
Progress: 600/5409
Progress: 700/5409
Progress: 800/5409
Progress: 900/5409
Progress: 1000/5409
Progress: 1100/5409
Progress: 1200/5409
Progress: 1300/5409
Progress: 1400/5409
Progress: 1500/5409
Progress: 1600/5409
Progress: 1700/5409
Progress: 1800/5409
Progress: 1900/5409
Progress: 2000/5409
Progress: 2100/5409
Progress: 2200/5409
Progress: 2300/5409
Progress: 2400/5409
Progress: 2500/5409
Progress: 2600/5409
Progress: 2700/5409
Progress: 2800/5409
Progress: 2900/5409
Progress: 3000/5409
Progress: 3100/5409
Progress: 3200/5409
Progress: 3300/5409
Progress: 3400/5409
Progress: 3500/5409
Progress: 3600/5409
Progress: 3700/5409
Progress: 3800/5409
Progress: 3900/5409
Progress: 4000/5409
Progress: 4100/5409
Progress: 4200/5409
Progress: 4300/5409
Progress: 4400/5409
Progress: 4500/5409
Progress: 4600/5409
Progress: 4700/5409
Progress: 4800/5409
Progress: 4900/5409
Progress: 5000/5409
Progress:

In [432]:
print(len(df3))
print(df3.head())

100000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  NewProduct FirstReviewTime  NewMarket  NewCategory  \
0                   1           1       4-09-2006          1            0   
1                   1           1       0-09-2006          1            0   
2                   1           1       5-05-2000          1            1   
3                   8           1       4-09-2006          1            0   
4                   8           1       4-09-2006          1            0   

   Imitation  
0        1.0  
1        0.0  
2        1.0  
3        1.0  
4        1.0  


In [435]:
from dateutil.relativedelta import relativedelta
from datetime import datetime
import pandas as pd

def parse_custom_date(date_str):
    # Extract the parts of the date
    day_of_week, month, year = map(int, date_str.split('-'))
    return datetime(year, month, 1)

def is_rating_declining(df, brewer_id, beer_style, review_date):
    three_weeks_ago = review_date - relativedelta(weeks=3)

    # Filter reviews for the same brewer and beer style within this period
    past_reviews = df[
        (df['Brewer ID'] == brewer_id) &
        (df['Beer Style'] == beer_style) &
        (pd.to_datetime(df['Time'], format='%w-%m-%Y') >= three_weeks_ago) &
        (pd.to_datetime(df['Time'], format='%w-%m-%Y') < review_date)
    ]

    if not past_reviews.empty:
        sorted_reviews = past_reviews.sort_values(by='Time')
        return sorted_reviews['Overall Review'].iloc[-1] < sorted_reviews['Overall Review'].iloc[0]
    return False

counter = 0
total_rows = len(df3)  # Calculate total rows in df3

def check_negative_performance(row, df, df3):
    global counter
    counter += 1

    if counter % 10 == 0:
        print(f"Processing row {counter} out of {total_rows}")

    review_date = parse_custom_date(row['Time'])
    month_start = review_date.replace(day=1)
    month_end = month_start + relativedelta(months=1) - relativedelta(days=1)

    # Identify competitor launches in the same month
    competitor_launches = df[
        (df['Brewer ID'] != row['Brewer ID']) &
        (pd.to_datetime(df['Time'], format='%w-%m-%Y') >= month_start) &
        (pd.to_datetime(df['Time'], format='%w-%m-%Y') <= month_end)
    ]

    if competitor_launches.empty:
        return 0

    # Check for shared 'Beer Style' and 'Rating First Digit'
    for _, competitor_row in competitor_launches.iterrows():
        if row['Beer Style'] == competitor_row['Beer Style'] and \
           row['Rating First Digit'] == competitor_row['Rating First Digit']:
            if is_rating_declining(df3, row['Brewer ID'], row['Beer Style'], review_date):
                return 1

    return 0

# Apply the function to create the 'Negative Performance - Dummy' column in df3
df3['NegativePerformanceD'] = df3.apply(lambda row: check_negative_performance(row, df, df3), axis=1)

print(f"Column NegativePerformanceD was successfully created. Total rows processed: {total_rows}")


Processing row 10 out of 100000
Processing row 20 out of 100000


KeyboardInterrupt: 

In [436]:
print(len(df3))
print(df3.head())

100000
   Brewer ID  Unnamed: 0       Time                    Beer Style  \
0       8481           0  4-09-2006  India Pale Ale &#40;IPA&#41;   
1       8481           1  0-09-2006  India Pale Ale &#40;IPA&#41;   
2       8481           2  5-05-2000             Bohemian Pilsener   
3       8481           3  4-09-2006                        Kölsch   
4       8481           4  4-09-2006                        Kölsch   

   Rating First Digit  NewProduct FirstReviewTime  NewMarket  NewCategory  \
0                   1           1       4-09-2006          1            0   
1                   1           1       0-09-2006          1            0   
2                   1           1       5-05-2000          1            1   
3                   8           1       4-09-2006          1            0   
4                   8           1       4-09-2006          1            0   

   Imitation  
0        1.0  
1        0.0  
2        1.0  
3        1.0  
4        1.0  


In [ ]:
########## Negative performance - continuous

# Merge 'Review - Overall' from df into df3
df3 = df3.merge(df[['Brewer ID', 'Time', 'Review - Overall']], 
                on=['Brewer ID', 'Time'], 
                how='left')

def calculate_competitors_average_rating(df, df3, review_date, beer_style, brewer_id):
   
    # Identify competitors who launched new market in the same month
    month_start = review_date.replace(day=1)
    month_end = month_start + relativedelta(months=1) - relativedelta(days=1)

    competitor_ids = df3[
        (df3['Time'].apply(lambda x: month_start <= datetime.strptime(x, '%w-%m-%Y') <= month_end)) &
        (df3['NewMarket'] == 1) &
        (df3['Brewer ID'] != brewer_id)
    ]['BrewerID'].unique()

    # Calculate the average 'Review - Overall' rating of these new products by competitors
    if competitor_ids.size > 0:
        competitor_new_products = df[
            (df['Brewer ID'].isin(competitor_ids)) &
            (df['Beer Style'] == beer_style) &
            (pd.to_datetime(df['Time'], format='%w-%m-%Y') <= review_date)
        ]
        if not competitor_new_products.empty:
            return competitor_new_products['Review - Overall'].mean()
    return 0

counter = 0  # Initialize counter
total_rows = len(df3)  # Total number of rows in df3

def calculate_negative_performance(row, df, df3):
    global counter
    counter += 1

    if counter % 10 == 0:
        print(f"Processing row {counter} out of {total_rows}")

    review_date = datetime.strptime(row['Time'], '%w-%m-%Y')
    # Remaining code for this function...

# Apply the function to create the 'Negative Performance' column in df3
df3['NegativePerformanceC'] = df3.apply(lambda row: calculate_negative_performance(row, df, df3), axis=1)

print(f"Column NegativePerformanceC was successfully created. Total rows processed: {total_rows}")



In [ ]:
########## CompetitorPosition

def calculate_competitors_average_rating_for_position(df, review_date, beer_style, brewer_id):
    # Calculate the average 'Review - Overall' rating of competitors up to the current review date
    competitor_ratings = df[
        (df['Brewer ID'] != brewer_id) &
        (df['Beer Style'] == beer_style) &
        (pd.to_datetime(df['Time'], format='%w-%m-%Y') <= review_date)
    ]

    if not competitor_ratings.empty:
        # Ensure 'Review - Overall' is treated as a numeric value
        competitor_ratings['Review - Overall'] = pd.to_numeric(competitor_ratings['Review - Overall'], errors='coerce')
        return competitor_ratings['Review - Overall'].mean()
    return 0

def calculate_competitor_position(row, df):
    review_date = datetime.strptime(row['Time'], '%w-%m-%Y')

    # Calculate competitor's average rating
    competitors_avg_rating = calculate_competitors_average_rating_for_position(df, review_date, row['Beer Style'], row['Brewer ID'])

    # Ensure 'Review - Overall' in df3 is numeric
    focal_firm_rating = pd.to_numeric(row['Review - Overall'], errors='coerce')

    # Compare competitor's average rating with the focal firm's rating
    return 1 if competitors_avg_rating > focal_firm_rating else 0

# Apply the function to create the 'CompetitorPosition' column in df3
df3['CompetitorPosition'] = df3.apply(lambda row: calculate_competitor_position(row, df), axis=1)


print("column CompetitorPosition was sucesfully created")

########## MRank 
########## SDRank 
########## Competition 
########## Portfolio 
########## ABV 
########## Diversification 
########## Tenure 
########## Experience 
########## Popularity 

In [ ]:
columns_to_keep = [
    'Brewer ID', 
    'Time', 
    'NewProduct', 
    'NewMarket', 
    'NewCategory', 
    'Imitation', 
    'NegativePerformanceD', 
    'NegativePerformanceC', 
    'CompetitorPosition'
]

df3 = df3[columns_to_keep]

print(df3.head())


   BrewerID       Time  NewProduct  NewMarket  NewCategory  Imitation  \
0      8481  4-09-2006           1          1            1          0   
1      8481  4-09-2006           1          1            1          0   
2      8481  4-09-2006           1          1            1          0   
3      8481  4-09-2006           1          1            1          0   
4      8481  4-09-2006           1          1            1          0   

   NegativePerformanceD  NegativePerformanceC  CompetitorPosition  
0                     0                     0                   0  
1                     0                     0                   0  
2                     0                     0                   0  
3                     0                     0                   0  
4                     0                     0                   0  
